### Colab에서 실행한다고 가정하고 작성함.

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 사용자 API키 변경 한 후 실행

In [3]:
import pandas as pd
import numpy as np
import torch
from openai import OpenAI
import torch.nn.functional as F
import pickle

# OpenAI 클라이언트 설정
client = OpenAI(
    api_key="사용자 API키",
    base_url="https://api.upstage.ai/v1/solar"
)

In [4]:
def get_embeddings(texts, batch_size=100):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        response = client.embeddings.create(
            input=batch_texts,
            model="solar-embedding-1-large-query"
        )
        batch_embeddings = [np.array(embedding.embedding) for embedding in response.data]
        embeddings.extend(batch_embeddings)
    return embeddings

### 사용자 경로 변경하고 실행하기

In [15]:
# 피클 파일로부터 딕셔너리을 불러오기
with open('사용자 경로', 'rb') as f:
    dic_read = pickle.load(f)

# 딕셔너리 전처리
menu_names = []
embeddings = []
texts = []

for key, value in dic_read.items():
    menu_names.append(key)
    embeddings.append(value[0])
    texts.append(value[1])

embeddings_np_array = np.array(embeddings)

In [16]:
# 사용자 입력 문장
queries = ["달콤하면서도 약간의 신맛이 조화를 이루어, 마치 여름 과일을 한입 베어 문 듯한 상쾌함을 느낄 수 있는 음식 추천해줘"]

# 코사인 유사도를 기준으로 입력 문장과 가장 유사한 5개 음식 추출
top_k = 5
for query in queries:
    query_embedding = get_embeddings([query])[0]
    query_embedding = torch.tensor(query_embedding)

    # 코사인 유사도 계산
    embeddings_tensor = torch.tensor(embeddings_np_array)
    cos_scores = F.cosine_similarity(query_embedding.unsqueeze(0), embeddings_tensor, dim=1)

    # 상위 top_k 결과 추출
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("사용자 입력 :", query)
    print("\nTop 5 음식 추천:")

    for idx in top_results.indices:
        print(menu_names[idx].strip()," : ",texts[idx].strip(),  "(Score: %.4f)" % (cos_scores[idx].item()))





사용자 입력 : 달콤하면서도 약간의 신맛이 조화를 이루어, 마치 여름 과일을 한입 베어 문 듯한 상쾌함을 느낄 수 있는 음식 추천해줘

Top 5 음식 추천:
토마토샐러드  :  떠먹는 토마토 샐러드는 토마토의 신선함과 소스의 달콤함이 너무 맛있답니다 (Score: 0.5817)
사과양배추샐러드  :  유튜브에서 보고 따라해봤어요.  상큼한게 맛있어요. (Score: 0.5642)
뚝배기배추국  :  얼큰하고 시원~한 맛이에요 (Score: 0.5439)
복숭아멘보샤  :  달콤한 데일리 복숭아가 탱글탱글 새우를 만나 튀김으로 하나가 되었다! 달콤 바삭한 복숭아 멘보샤  자칫하면 느끼할 수 있는 튀김요리의 신의 한 수  '새콤달콤 복숭아칠리소스와 팬에 구워 달콤함이 배가된 복숭아'를 곁들여 먹는 맛난 간식! 한입 크기로 파티 핑거푸드로도 좋아요~ (Score: 0.5418)
찜닭  :  달달한 간장양념이 가미된 찜닭요리를 해 봤어요! :) 여름보양식으로 딱이예요. (Score: 0.5381)
